# [6] Wardバッチ処理 - Phase 3

ミニマップキャプチャからward座標を抽出し、タイムラインデータと統合します。

## 処理フロー
1. **YOLO推論**: 全フレームでward検出（GPUバッチ推論対応）
2. **クラスタリング**: 同一wardをグループ化
3. **タイムライン統合**: Riot APIのwardイベントとマッチング

## 入出力
- **入力**: ミニマップ画像 (`C:\dataset_20260105\JP1-*\0\*.png`)
- **出力**: 
  - `detections_raw.csv` - 生の検出結果
  - `wards.csv` - クラスタリング後のward情報
  - `wards_matched.csv` - タイムライン統合済み

## 前提条件
- Phase 2完了（YOLOv8モデル `models/best.pt`）
- ミニマップキャプチャ完了（notebook 04）
- タイムラインデータ（`data/timeline/*.json`）

In [ ]:
#cell-1: セットアップ（インポート + 設定 + モデル読み込み）
from pathlib import Path
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from dataclasses import dataclass, field
from typing import List, Dict
import time

from tqdm import tqdm
from ultralytics import YOLO
import torch

# プロジェクトルート
PROJECT_ROOT = Path(r"c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2")

import sys
sys.path.insert(0, str(PROJECT_ROOT))

from autoLeague.dataset.ward_tracker import WardTracker

# === パス設定 ===
MODEL_PATH = PROJECT_ROOT / "models" / "best.pt"
DATASET_DIR = Path(r"C:\dataset_20260105")
TIMELINE_DIR = PROJECT_ROOT / "data" / "timeline"

# === 推論設定 ===
CONFIDENCE_THRESHOLD = 0.6
IMAGE_SIZE = 512

# GPUバッチサイズ（VRAM容量に応じて自動調整）
# RTX 3050 (4GB): 8, RTX 3080 (10GB): 32, A100 (40GB): 64
if torch.cuda.is_available():
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    BATCH_SIZE = 8 if vram_gb < 6 else (32 if vram_gb < 16 else 64)
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {vram_gb:.1f} GB")
else:
    BATCH_SIZE = 1
    print("GPU: 利用不可（CPUで実行）")

# === クラスタリング設定 ===
DISTANCE_THRESHOLD = 0.01  # 同一wardと判定する座標距離
MIN_FRAMES = 3  # ノイズ除去：最小連続フレーム数
GAP_TOLERANCE = 10  # 検出が途切れても同一wardとみなすフレーム数

# === モデル読み込み ===
model = YOLO(str(MODEL_PATH))

print(f"\nバッチサイズ: {BATCH_SIZE}")
print(f"モデル: {MODEL_PATH.name}")
print(f"クラス: {model.names}")
print(f"\nデータセット: {DATASET_DIR}")
if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    print(f"試合数: {len(match_dirs)}")

In [ ]:
#cell-2: 関数定義（データ構造 + 推論 + クラスタリング）

@dataclass
class Detection:
    """1フレームでの検出結果"""
    frame: int
    class_id: int
    class_name: str
    x: float  # 正規化座標 (0-1)
    y: float
    w: float
    h: float
    confidence: float


@dataclass
class Ward:
    """クラスタリング後のward"""
    ward_id: int
    class_id: int
    class_name: str
    x: float
    y: float
    frame_start: int
    frame_end: int
    detections: List[Detection] = field(default_factory=list)

    @property
    def confidence_avg(self) -> float:
        return sum(d.confidence for d in self.detections) / len(self.detections) if self.detections else 0.0

    @property
    def detection_count(self) -> int:
        return len(self.detections)


def run_inference(model: YOLO, match_dir: Path, conf: float = CONFIDENCE_THRESHOLD,
                  batch_size: int = BATCH_SIZE) -> List[Detection]:
    """1試合分の全フレームをバッチ推論"""
    frame_dir = match_dir / "0"
    if not frame_dir.exists():
        return []

    frame_files = sorted(frame_dir.glob("*.png"), key=lambda p: int(p.stem))
    if not frame_files:
        return []

    detections: List[Detection] = []
    total_frames = len(frame_files)

    for i in range(0, total_frames, batch_size):
        batch_files = frame_files[i:i+batch_size]
        batch_paths = [str(p) for p in batch_files]
        frame_nums = [int(p.stem) for p in batch_files]

        results = model(batch_paths, imgsz=IMAGE_SIZE, conf=conf, verbose=False)

        for frame_num, result in zip(frame_nums, results):
            for box in result.boxes:
                class_id = int(box.cls[0])
                class_name = model.names[class_id]
                x, y, w, h = box.xywhn[0].tolist()
                confidence = float(box.conf[0])
                detections.append(Detection(
                    frame=frame_num, class_id=class_id, class_name=class_name,
                    x=x, y=y, w=w, h=h, confidence=confidence
                ))

    return detections


def cluster_detections(detections: List[Detection]) -> List[Ward]:
    """検出結果をクラスタリングしてward単位にまとめる"""
    if not detections:
        return []

    sorted_detections = sorted(detections, key=lambda d: d.frame)
    active_wards: Dict[int, List[Ward]] = defaultdict(list)
    completed_wards: List[Ward] = []
    next_ward_id = 1

    for det in sorted_detections:
        matched = False
        for ward in active_wards[det.class_id]:
            dist = np.sqrt((det.x - ward.detections[-1].x)**2 + (det.y - ward.detections[-1].y)**2)
            if dist < DISTANCE_THRESHOLD and det.frame - ward.frame_end <= GAP_TOLERANCE:
                ward.detections.append(det)
                ward.frame_end = det.frame
                ward.x = sum(d.x for d in ward.detections) / len(ward.detections)
                ward.y = sum(d.y for d in ward.detections) / len(ward.detections)
                matched = True
                break

        if not matched:
            new_ward = Ward(
                ward_id=next_ward_id, class_id=det.class_id, class_name=det.class_name,
                x=det.x, y=det.y, frame_start=det.frame, frame_end=det.frame, detections=[det]
            )
            active_wards[det.class_id].append(new_ward)
            next_ward_id += 1

        current_frame = det.frame
        for class_id in list(active_wards.keys()):
            still_active = []
            for ward in active_wards[class_id]:
                if current_frame - ward.frame_end > GAP_TOLERANCE:
                    completed_wards.append(ward)
                else:
                    still_active.append(ward)
            active_wards[class_id] = still_active

    for class_id in active_wards:
        completed_wards.extend(active_wards[class_id])

    filtered_wards = [w for w in completed_wards if w.detection_count >= MIN_FRAMES]
    for i, ward in enumerate(sorted(filtered_wards, key=lambda w: w.frame_start), start=1):
        ward.ward_id = i

    return sorted(filtered_wards, key=lambda w: w.frame_start)


def save_detections(detections: List[Detection], output_path: Path):
    """生の検出結果をCSVに保存"""
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['frame', 'class_id', 'class_name', 'x', 'y', 'w', 'h', 'confidence'])
        for d in detections:
            writer.writerow([d.frame, d.class_id, d.class_name,
                           f"{d.x:.6f}", f"{d.y:.6f}", f"{d.w:.6f}", f"{d.h:.6f}",
                           f"{d.confidence:.4f}"])


def save_wards(wards: List[Ward], output_path: Path):
    """クラスタリング後のward情報をCSVに保存"""
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['ward_id', 'class_id', 'class_name', 'x', 'y',
                        'frame_start', 'frame_end', 'detection_count', 'confidence_avg'])
        for w in wards:
            writer.writerow([w.ward_id, w.class_id, w.class_name,
                           f"{w.x:.6f}", f"{w.y:.6f}",
                           w.frame_start, w.frame_end,
                           w.detection_count, f"{w.confidence_avg:.4f}"])

print("関数定義完了")

## オプション: テスト推論

1試合100フレームでテスト推論を実行し、処理速度を確認します。  
本番実行時はスキップ可能です。

In [ ]:
#cell-3: テスト推論（オプション）
match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
if match_dirs:
    test_match = match_dirs[0]
    frame_dir = test_match / "0"
    frame_files = sorted(frame_dir.glob("*.png"), key=lambda p: int(p.stem))[:100]

    print(f"テスト: {test_match.name} (100フレーム)")
    print(f"バッチサイズ: {BATCH_SIZE}")

    start_time = time.time()
    test_detections = []

    for i in range(0, len(frame_files), BATCH_SIZE):
        batch_files = frame_files[i:i+BATCH_SIZE]
        batch_paths = [str(p) for p in batch_files]
        frame_nums = [int(p.stem) for p in batch_files]
        results = model(batch_paths, imgsz=IMAGE_SIZE, conf=CONFIDENCE_THRESHOLD, verbose=False)
        for frame_num, result in zip(frame_nums, results):
            for box in result.boxes:
                test_detections.append(Detection(
                    frame=frame_num, class_id=int(box.cls[0]), class_name=model.names[int(box.cls[0])],
                    x=box.xywhn[0][0].item(), y=box.xywhn[0][1].item(),
                    w=box.xywhn[0][2].item(), h=box.xywhn[0][3].item(),
                    confidence=float(box.conf[0])
                ))

    elapsed = time.time() - start_time
    fps = len(frame_files) / elapsed

    test_wards = cluster_detections(test_detections)

    print(f"\n処理速度: {fps:.1f} FPS")
    print(f"検出数: {len(test_detections)} -> ward数: {len(test_wards)}")

## 全試合バッチ処理

YOLO推論 -> クラスタリング -> タイムライン統合を1ループで実行します。

In [ ]:
#cell-4: 全試合バッチ処理（YOLO推論 + クラスタリング + タイムライン統合）
match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
total_matches = len(match_dirs)

# WardTracker初期化（ハンガリアン法使用）
tracker = WardTracker(
    timeline_dir=TIMELINE_DIR,
    dataset_dir=DATASET_DIR,
    use_hungarian=True,
)

print(f"全{total_matches}試合の処理を開始")
print(f"バッチサイズ: {BATCH_SIZE}")
print("="*60)

# 累積統計
cumulative_matched = 0
cumulative_total = 0
results = []
start_time = time.time()

for i, match_dir in enumerate(tqdm(match_dirs, desc="全体進捗")):
    match_id = match_dir.name

    try:
        # 1. YOLO推論
        detections = run_inference(model, match_dir)
        if not detections:
            results.append({"match": match_id, "detections": 0, "wards": 0, "matched": 0})
            continue

        # 2. 生の検出結果を保存
        save_detections(detections, match_dir / "detections_raw.csv")

        # 3. クラスタリング
        wards = cluster_detections(detections)
        save_wards(wards, match_dir / "wards.csv")

        # 4. タイムライン統合
        matched_wards = tracker.process_match(match_id)

        # 統計計算
        matched_count = sum(1 for w in matched_wards if w.match_status == "matched")
        total_count = len([w for w in matched_wards if w.match_status != "detection_only"])

        cumulative_matched += matched_count
        cumulative_total += total_count
        match_rate = cumulative_matched / cumulative_total * 100 if cumulative_total > 0 else 0

        results.append({
            "match": match_id,
            "detections": len(detections),
            "wards": len(wards),
            "matched": matched_count
        })

        # 進捗表示（10試合ごと）
        if (i + 1) % 10 == 0 or i == total_matches - 1:
            tqdm.write(f"[{i+1}/{total_matches}] 累積マッチング率: {match_rate:.1f}%")

    except Exception as e:
        tqdm.write(f"エラー [{match_id}]: {e}")
        results.append({"match": match_id, "detections": 0, "wards": 0, "matched": 0, "error": str(e)})

total_time = time.time() - start_time

print("\n" + "="*60)
print("処理完了")
print("="*60)
print(f"処理試合数: {len(results)}")
print(f"総検出数: {sum(r['detections'] for r in results)}")
print(f"総ward数: {sum(r['wards'] for r in results)}")
print(f"マッチング率: {cumulative_matched}/{cumulative_total} ({match_rate:.1f}%)")
print(f"総処理時間: {total_time/60:.1f}分")

In [ ]:
#cell-5: 結果確認
print("=== 処理結果サマリー ===")

# 各試合のwards_matched.csvを確認
match_results = []
for match_dir in match_dirs:
    matched_csv = match_dir / "wards_matched.csv"
    if matched_csv.exists():
        df = pd.read_csv(matched_csv)
        match_results.append({
            "match": match_dir.name,
            "wards": len(df),
            "matched": len(df[df["match_status"] == "matched"]) if "match_status" in df.columns else 0,
        })

results_df = pd.DataFrame(match_results)
print(f"\n処理済み試合: {len(results_df)}")
print(f"総ward数: {results_df['wards'].sum()}")
print(f"マッチ済み: {results_df['matched'].sum()}")

if len(results_df) > 0 and results_df['wards'].sum() > 0:
    final_match_rate = results_df['matched'].sum() / results_df['wards'].sum() * 100
    print(f"最終マッチング率: {final_match_rate:.1f}%")

display(results_df.head(10))

In [ ]:
#cell-6: グリッド特徴量生成（ward_grid.npz）
from autoLeague.scoring.grid_generator import generate_ward_grid, save_ward_grid

print("グリッド特徴量生成を開始...")
print("="*60)

grid_success = 0
grid_skip = 0

for match_dir in tqdm(match_dirs, desc="グリッド生成"):
    wards_csv = match_dir / "wards_matched.csv"
    output_path = match_dir / "ward_grid.npz"
    
    if not wards_csv.exists():
        grid_skip += 1
        continue
    
    try:
        grid_data = generate_ward_grid(wards_csv)
        save_ward_grid(grid_data, output_path)
        grid_success += 1
    except Exception as e:
        tqdm.write(f"エラー [{match_dir.name}]: {e}")
        grid_skip += 1

print("\n" + "="*60)
print(f"グリッド生成完了: {grid_success}試合")
print(f"スキップ: {grid_skip}試合")

In [ ]:
#cell-7: 次のステップ
print("="*60)
print("Phase 3 完了")
print("="*60)
print("\n出力ファイル:")
print("  - detections_raw.csv: 生の検出結果")
print("  - wards.csv: クラスタリング後ward情報")
print("  - wards_matched.csv: タイムライン統合済み")
print("  - ward_grid.npz: グリッド特徴量（32x32x3時間帯）")
print("\n次のステップ:")
print("  -> 07_vision_score.ipynb でモデル学習・ヒートマップ可視化")